## Exercícios

> Retirados de [learn-python: sqlalchemy_orm-questions](https://aviadr1.github.io/learn-advanced-python/11_db_access/exercise/sqlalchemy_orm-questions.html).

#### Q1.

Baixa e extraia o arquivo compactado com o banco de dados [Chinook database](https://www.sqlitetutorial.net/sqlite-sample-database/). Salve o arquivo `chinook.db` na mesma pasta deste script.
* Link para baixar: http://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip

<img width=500 src=https://www.sqlitetutorial.net/wp-content/uploads/2015/11/sqlite-sample-database-color.jpg>


#### Q2.

Leia o código e os comentários das células a seguir para entender como acessamos os modelos ORM de um banco já existente.

In [ ]:
from sqlalchemy import create_engine, text, MetaData
from sqlalchemy.orm import Session

engine = create_engine("sqlite+pysqlite:///chinook.db", echo=False)

### extrai as classes da base de dados Chinook
metadata = MetaData()
metadata.reflect(engine)

# O metadata tem informações sobre as tabelas
# que serão usadas para criar os modelos ORM
for table_name, table in metadata.tables.items():
    print(table_name)
    print(table.columns.keys())
    print(table.columns.items())
    print('-'*25)

### configura o objeto Base mapeando os modelos ORM das tabelas
from sqlalchemy.ext.automap import automap_base
Base = automap_base(metadata=metadata)
Base.prepare()

# o objeto Base tem os modelos ORM que podemos usar
# para manipular o banco de dados
print(Base.classes.items())

In [ ]:
# A seguir um exemplo de query na tabela Albums
# usamos o objeto Base para acessar cada modelo ORM.

session = Session(engine)
res = session.scalars(select(Base.classes.albums))
first_album = res.first()
print(first_album.AlbumId, first_album.Title)

#### Q3. 
Com base nos códigos anteriores realize as operações solicitadas nas células a seguir:


In [1]:
from sqlalchemy import create_engine, text, MetaData, select
from sqlalchemy.orm import Session
from sqlalchemy.ext.automap import automap_base

engine = create_engine("sqlite+pysqlite:///chinook.db", echo=False)
metadata = MetaData()
metadata.reflect(engine)
Base = automap_base(metadata=metadata)
Base.prepare()
session = Session(engine)

# Imprime os três primeiros registros da tabela tracks
tracks_table = Base.classes.tracks
result = session.scalars(select(tracks_table).limit(3))
for track in result:
    print(f"TrackId: {track.TrackId}, Name: {track.Name}, AlbumId: {track.AlbumId}")

TrackId: 1, Name: For Those About To Rock (We Salute You), AlbumId: 1
TrackId: 2, Name: Balls to the Wall, AlbumId: 2
TrackId: 3, Name: Fast As a Shark, AlbumId: 3


In [2]:
from sqlalchemy import join

# Imprime o nome da faixa e o título do álbum das primeiras 20 faixas na tabela tracks.
tracks_table = Base.classes.tracks
albums_table = Base.classes.albums

result = session.execute(
    select(tracks_table.Name, albums_table.Title)
    .join(albums_table, tracks_table.AlbumId == albums_table.AlbumId)
    .limit(20)
)
for track_name, album_title in result:
    print(f"Faixa: {track_name} - Álbum: {album_title}")


Faixa: For Those About To Rock (We Salute You) - Álbum: For Those About To Rock We Salute You
Faixa: Put The Finger On You - Álbum: For Those About To Rock We Salute You
Faixa: Let's Get It Up - Álbum: For Those About To Rock We Salute You
Faixa: Inject The Venom - Álbum: For Those About To Rock We Salute You
Faixa: Snowballed - Álbum: For Those About To Rock We Salute You
Faixa: Evil Walks - Álbum: For Those About To Rock We Salute You
Faixa: C.O.D. - Álbum: For Those About To Rock We Salute You
Faixa: Breaking The Rules - Álbum: For Those About To Rock We Salute You
Faixa: Night Of The Long Knives - Álbum: For Those About To Rock We Salute You
Faixa: Spellbound - Álbum: For Those About To Rock We Salute You
Faixa: Balls to the Wall - Álbum: Balls to the Wall
Faixa: Fast As a Shark - Álbum: Restless and Wild
Faixa: Restless and Wild - Álbum: Restless and Wild
Faixa: Princess of the Dawn - Álbum: Restless and Wild
Faixa: Go Down - Álbum: Let There Be Rock
Faixa: Dog Eat Dog - Álbum: Le

In [3]:
# Imprime as 10 primeiras vendas de faixas da tabela invoice_items
invoice_items_table = Base.classes.invoice_items
result_sales = session.scalars(select(invoice_items_table).limit(10))
for item in result_sales:
    print(f"Venda: {item.InvoiceLineId}, FaixaId: {item.TrackId}, Quantidade: {item.Quantity}")

print("-" * 25)

# Para essas 10 primeiras vendas, imprima os nomes das faixas vendidas e a quantidade vendida.
tracks_table = Base.classes.tracks
result_sales_with_names = session.execute(
    select(tracks_table.Name, invoice_items_table.Quantity)
    .join(invoice_items_table, invoice_items_table.TrackId == tracks_table.TrackId)
    .limit(10)
)
for track_name, quantity in result_sales_with_names:
    print(f"Nome da Faixa: {track_name}, Quantidade: {quantity}")


Venda: 1, FaixaId: 2, Quantidade: 1
Venda: 2, FaixaId: 4, Quantidade: 1
Venda: 3, FaixaId: 6, Quantidade: 1
Venda: 4, FaixaId: 8, Quantidade: 1
Venda: 5, FaixaId: 10, Quantidade: 1
Venda: 6, FaixaId: 12, Quantidade: 1
Venda: 7, FaixaId: 16, Quantidade: 1
Venda: 8, FaixaId: 20, Quantidade: 1
Venda: 9, FaixaId: 24, Quantidade: 1
Venda: 10, FaixaId: 28, Quantidade: 1
-------------------------
Nome da Faixa: Balls to the Wall, Quantidade: 1
Nome da Faixa: Restless and Wild, Quantidade: 1
Nome da Faixa: Put The Finger On You, Quantidade: 1
Nome da Faixa: Inject The Venom, Quantidade: 1
Nome da Faixa: Evil Walks, Quantidade: 1
Nome da Faixa: Breaking The Rules, Quantidade: 1
Nome da Faixa: Dog Eat Dog, Quantidade: 1
Nome da Faixa: Overdose, Quantidade: 1
Nome da Faixa: Love In An Elevator, Quantidade: 1
Nome da Faixa: Janie's Got A Gun, Quantidade: 1


In [4]:
from sqlalchemy import func

# Imprima os nomes das 10 faixas mais vendidas e quantas vezes foram vendidas.
tracks_table = Base.classes.tracks
invoice_items_table = Base.classes.invoice_items

result = session.execute(
    select(tracks_table.Name, func.sum(invoice_items_table.Quantity).label('total_sales'))
    .join(invoice_items_table, invoice_items_table.TrackId == tracks_table.TrackId)
    .group_by(tracks_table.Name)
    .order_by(func.sum(invoice_items_table.Quantity).desc())
    .limit(10)
)
for track_name, total_sales in result:
    print(f"Faixa: {track_name}, Total de vendas: {total_sales}")


Faixa: The Trooper, Total de vendas: 5
Faixa: Untitled, Total de vendas: 4
Faixa: The Number Of The Beast, Total de vendas: 4
Faixa: Sure Know Something, Total de vendas: 4
Faixa: Hallowed Be Thy Name, Total de vendas: 4
Faixa: Eruption, Total de vendas: 4
Faixa: Where Eagles Dare, Total de vendas: 3
Faixa: Welcome Home (Sanitarium), Total de vendas: 3
Faixa: Sweetest Thing, Total de vendas: 3
Faixa: Surrender, Total de vendas: 3


In [5]:
# Quem são os 10 artistas que mais venderam?
invoice_items_table = Base.classes.invoice_items
tracks_table = Base.classes.tracks
albums_table = Base.classes.albums
artists_table = Base.classes.artists

result = session.execute(
    select(artists_table.Name, func.sum(invoice_items_table.Quantity).label('total_sales'))
    .join(tracks_table, tracks_table.TrackId == invoice_items_table.TrackId)
    .join(albums_table, albums_table.AlbumId == tracks_table.AlbumId)
    .join(artists_table, artists_table.ArtistId == albums_table.ArtistId)
    .group_by(artists_table.Name)
    .order_by(func.sum(invoice_items_table.Quantity).desc())
    .limit(10)
)

for artist_name, total_sales in result:
    print(f"Artista: {artist_name}, Total de vendas: {total_sales}")

session.close()


Artista: Iron Maiden, Total de vendas: 140
Artista: U2, Total de vendas: 107
Artista: Metallica, Total de vendas: 91
Artista: Led Zeppelin, Total de vendas: 87
Artista: Os Paralamas Do Sucesso, Total de vendas: 45
Artista: Deep Purple, Total de vendas: 44
Artista: Faith No More, Total de vendas: 42
Artista: Lost, Total de vendas: 41
Artista: Eric Clapton, Total de vendas: 40
Artista: R.E.M., Total de vendas: 39
